In [38]:
## Importing packages that will be used ##
import numpy as np
import pandas as pd
import csv



In [106]:
### Parsing the Hurdat2 Dataset ###
storms = []
with open('data/hurdat2_8_28_25.csv') as hurdat2:
    info = hurdat2.readlines()
i = 0
while i < len(info):
    line = info[i].strip().split(',')
    storm_id = line[0].strip()
    storm_name = line[1].strip()
    track_entries = int(line[2].strip())
    year = storm_id[-4:]

    tracks = [info[j].strip().split(',') for j in range(i+1, i+1+track_entries)]
    df = pd.DataFrame(tracks, columns = ["date", "time", "ID", "Cylcone Type", "Lat", "lon",
                                              "Max Winds", "Min Pressure", "34kt NEQ", "34kt SEQ", 
                                              "34kt SWQ", "34kt NWQ", "50kt NEQ", "50kt SEQ", "50kt SWQ",
                                              "50kt NWQ", "64kt NEQ", "64kt SEQ","64kt SWQ", "64kt NWQ", 
                                              "Radius of Max Winds"])
    df["storm_id"] = storm_id
    df["storm_name"] = storm_name
    df["year"] = year

    if any(id.strip() == "L" for id in df["ID"].values):
        storms.append(df)
    i += track_entries + 1
       
Landfalling_storms = pd.concat(storms, ignore_index = True)



In [107]:
### Just seeing the Datframe ###
Landfalling_storms

,date,time,ID,Cylcone Type,Lat,lon,Max Winds,Min Pressure,34kt NEQ,34kt SEQ,...,50kt SWQ,50kt NWQ,64kt NEQ,64kt SEQ,64kt SWQ,64kt NWQ,Radius of Max Winds,storm_id,storm_name,year
0,18510625,0000,,HU,28.0N,94.8W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,AL011851,UNNAMED,1851
1,18510625,0600,,HU,28.0N,95.4W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,AL011851,UNNAMED,1851
2,18510625,1200,,HU,28.0N,96.0W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,AL011851,UNNAMED,1851
3,18510625,1800,,HU,28.1N,96.5W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,AL011851,UNNAMED,1851
4,18510625,2100,L,HU,28.2N,96.8W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,AL011851,UNNAMED,1851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22832,20241117,0600,,TS,16.5N,87.5W,35,1001,80,30,...,0,0,0,0,0,0,40,AL192024,SARA,2024
22833,20241117,1200,,TS,16.8N,87.9W,35,1001,80,30,...,0,0,0,0,0,0,40,AL192024,SARA,2024
22834,20241117,1400,L,TS,17.0N,88.3W,35,1001,80,30,...,0,0,0,0,0,0,40,AL192024,SARA,2024
22835,20241117,1800,,TD,17.4N,89.1W,30,1003,0,0,...,0,0,0,0,0,0,50,AL192024,SARA,2024


In [115]:
### Altering the Dataframe ###
Landfall_only = Landfalling_storms[Landfalling_storms["ID"].str.strip() == "L"] # Showing only the landfalls for each storm
Landfall_only = Landfall_only.set_index(['storm_name', 'year']) ## Setting a multi-index with storm name and year
Landfall_only = Landfall_only.drop(columns = ['ID', 'storm_id']) # Dropping ID and Storm_id cause they are no longer needed



In [116]:
Landfall_only

date   time Cylcone Type     Lat      lon Max Winds  \
storm_name year                                                            
UNNAMED    1851  18510625   2100           HU   28.2N    96.8W        80   
           1851  18510823   2100           HU   30.1N    85.7W       100   
           1851  18511019   1500           TS   41.1N    71.7W        50   
           1852  18520826   0600           HU   30.2N    88.6W       100   
           1852  18520912   0000           HU   28.0N    82.8W        70   
...                   ...    ...          ...     ...      ...       ...   
OSCAR      2024  20241019   2015           HU   21.5N    71.1W        75   
           2024  20241020   2200           HU   20.3N    74.4W        75   
RAFAEL     2024  20241106   2115           HU   22.7N    82.7W       100   
SARA       2024  20241115   0120           TS   15.8N    84.3W        40   
           2024  20241117   1400           TS   17.0N    88.3W        35   

                Min Pressure 34kt NEQ 34kt SEQ 34kt SWQ 34kt NWQ 50kt NEQ  \
storm_name year                                                             
UNNAMED    1851         -999     -999     -999     -999     -999     -999   
           1851         -999     -999     -999     -999     -999     -999   
           1851         -999     -999     -999     -999     -999     -999   
           1852          961     -999     -999     -999     -999     -999   
           1852         -999     -999     -999     -999     -999     -999   
...                      ...      ...      ...      ...      ...      ...   
OSCAR      2024          987       40       30       30       40       20   
           2024          984       40       30       30       40       20   
RAFAEL     2024          955      100       90       40       50       50   
SARA       2024          999       90        0        0       90        0   
           2024         1001       80       30       30       60        0   

                50kt SEQ 50kt SWQ 50kt NWQ 64kt NEQ 64kt SEQ 64kt SWQ  \
storm_name year                                                         
UNNAMED    1851     -999     -999     -999     -999     -999     -999   
           1851     -999     -999     -999     -999     -999     -999   
           1851     -999     -999     -999     -999     -999     -999   
           1852     -999     -999     -999     -999     -999     -999   
           1852     -999     -999     -999     -999     -999     -999   
...                  ...      ...      ...      ...      ...      ...   
OSCAR      2024       10       10       20        5        5        0   
           2024       10       10       20       10        5        5   
RAFAEL     2024       40       20       25       25       20       10   
SARA       2024        0        0        0        0        0        0   
           2024        0        0        0        0        0        0   

                64kt NWQ Radius of Max Winds  
storm_name year                               
UNNAMED    1851     -999                -999  
           1851     -999                -999  
           1851     -999                -999  
           1852     -999                  10  
           1852     -999                -999  
...                  ...                 ...  
OSCAR      2024        5                   5  
           2024       10                   5  
RAFAEL     2024       15                  10  
SARA       2024        0                  40  
           2024        0                  40  

[1167 rows x 20 columns]